## Importing Packages
The first part of the assignment is importing the packages

In [ ]:
import (
	"encoding/csv"
    "gonum.org/v1/gonum/floats"
	"fmt"
	"log"
	"os"
	"sort"
	"strconv"
    "context"
	"io/ioutil"
	"log"
	"strings"
)

## Main Function
Next, we define this main function. You don't need to do anything here.<br>
This function calls in the other functions to calculate recommendations based on the "Ideal Location", a row in our dataset, and print out a top 5. **But, main won't run!**<br>
Because the other functions are not completed. Your job is to write these other functions! Save the Gophers by writing these other functions first and then running them, before running main().

In [ ]:
func main() {
    // Load the data
    records := loadData()
    
    // Parse the records into locations
    locations := parseRecords(records)

    // Recommend locations based on a given location
    recommendations := recommendBasedOnLocation(locations, "Ideal Location")
	if recommendations != nil {
		fmt.Println("Top recommended locations:")
		for _, rec := range recommendations[:5] {
			fmt.Printf("%s (Similarity: %.2f)\n", rec.Location.Name, rec.Similarity)
		}
	}
}

## Load Data
The first step: Loading the data.

In [ ]:
// Open the CSV file and create a new CSV reader reading from the opened file
func loadData() [][]string {
    // Open the CSV file
    f, err := os.Open("gopher_locations.csv")
    if err != nil {
        log.Fatalf("Cannot open 'locations.csv': %s\n", err.Error())
    }
    defer f.Close()

    // Create a new CSV reader reading from the opened file
    reader := csv.NewReader(f)

    // Read all the records from the CSV file
    records, err := reader.ReadAll()
    if err != nil {
        log.Fatalf("Cannot read CSV data: %s\n", err.Error())
    }

    return records
}

## Define Location struct

Next up is defining a struct of the data in each row, a Location struct. We need that, because when we define the data into a struct, we can more easily access all its properties. <br>
These are the columns that are in the dataset, and should be represented in the Location struct. <br>
After the : is the Datatype that we recommend for each content, in order to be able to make vectors from them for the recommendation. <br>
Don't worry, we're going to convert all the data to those datatypes after this.<br>

1) Name: Name of the location : string
2) AverageTemperature: Average temperature of the location : float64
3) NrPredators: Number of predators in the location : float64
4) NrFoodSources: Number of food sources in the location : float64
5) NrWaterSources: Number of water sources in the location : float64
6) NrHumans: Number of humans in the location : float64
7) VegetationType: Type of vegetation in the location : float64
8) LandSize: Size of the location : float64

In [ ]:
// Define a structure to hold location data
type Location struct {
	Name             string
	AverageTemperature      float64
	NrPredators  float64
	NrFoodSources float64
	NrWaterSources     float64
	NrHumans    float64
	VegetationType       float64
	LandSize         float64
}

## Parse the Records

Next up, parsing the records that we got from the CSV, converting the values that were all loaded as a string into a float64, so we can make vectors from them, and then passing them into a Location struct. <br>
We will give you a helper function below. This returns a float from a string-float-map by inputting the key string. Good for categorical values. *wink wink*. The map, however, needs to be created first, in the parseRecords method. 

In [ ]:
// Helper function to encode categorical variables for VegetationType column
func encodeCategory(value string, categories map[string]float64) float64 {
	return categories[value]
}

In [ ]:
// Parse the records into locations
func parseRecords(records [][]string) []Location {
    locations := []Location{}
    vegetationCategories := map[string]float64{"Fields": 0, "Meadow": 1, "Hills": 2, "Forest": 3, "Wetland": 4, "Savanna": 5, "Desert": 6}

    for _, record := range records[1:] { // Skip header
        averageTemp, _ := strconv.ParseFloat(record[1], 64)
        nrPredators, _ := strconv.ParseFloat(record[2], 64)
        nrFoodSources, _ := strconv.ParseFloat(record[3], 64)
        nrWaterSources, _ := strconv.ParseFloat(record[4],64)
        nrHumans, _ := strconv.ParseFloat(record[5],64)
        vegetationType := encodeCategory(record[6], vegetationCategories)
        landSize, _ := strconv.ParseFloat(record[7], 64)

        location := Location{
            Name:             record[0],
            AverageTemperature:      averageTemp,
            NrPredators:  nrPredators,
            NrFoodSources: nrFoodSources,
            NrWaterSources:     nrWaterSources,
            NrHumans:    nrHumans,
            VegetationType:       vegetationType,
            LandSize:         landSize,
        }
        locations = append(locations, location)
    }
    return locations
}

## Calculate Similarity
Next, we have this super-complicated function that calculates the cosine similarity between two vectors (or []float64s). <br>
Credits to gaspiman, if you read this mate, your package broke, so we stole your code. <br>
You don't have to do anything with this, thankfully we didn't either, just run it so you can call it in the next function.

In [ ]:
func Cosine(a []float64, b []float64) (cosine float64, err error) {
	count := 0
	length_a := len(a)
	length_b := len(b)
	if length_a > length_b {
		count = length_a
	} else {
		count = length_b
	}
	sumA := 0.0
	s1 := 0.0
	s2 := 0.0
	for k := 0; k < count; k++ {
		if k >= length_a {
			s2 += math.Pow(b[k], 2)
			continue
		}
		if k >= length_b {
			s1 += math.Pow(a[k], 2)
			continue
		}
		sumA += a[k] * b[k]
		s1 += math.Pow(a[k], 2)
		s2 += math.Pow(b[k], 2)
	}
	if s1 == 0 || s2 == 0 {
		return 0.0, errors.New("Vectors should not be null (all zeros)")
	}
	return sumA / (math.Sqrt(s1) * math.Sqrt(s2)), nil
}

In this next function, we calculate the similarity between two Locations. This is done in 3 steps:
- Define the max range per feature (we did that one for you)
- Normalize the features of both Locations. If you put these values into an array of float64, that's like a vector of points.
- Calculate the similarity between the two vectors using the Cosine function.

In [ ]:
// Function to calculate similarity score
func calculateSimilarity(ideal Location, other Location) float64 {
	// Define the ranges of the features
	ranges := map[string]float64{
		"AverageTemperature":  40.0,
		"NrPredators":       20.0,
		"NrFoodSources":       20.0,
		"NrWaterSources":      10.0,
		"NrHumans":            50.0,
		"VegetationType":      6.0,
		"LandSize":            1000.0,
	}

	// Normalize the features
	idealVector := []float64{
		ideal.AverageTemperature / ranges["AverageTemperature"],
		ideal.NrPredators / ranges["NrPredators"],
		ideal.NrFoodSources / ranges["NrFoodSources"],
		ideal.NrWaterSources / ranges["NrWaterSources"],
		ideal.NrHumans / ranges["NrHumans"],
		ideal.VegetationType / ranges["VegetationType"],
		ideal.LandSize / ranges["LandSize"],
	}
	otherVector := []float64{
		other.AverageTemperature / ranges["AverageTemperature"],
		other.NrPredators / ranges["NrPredators"],
		other.NrFoodSources / ranges["NrFoodSources"],
		other.NrWaterSources / ranges["NrWaterSources"],
		other.NrHumans / ranges["NrHumans"],
		other.VegetationType / ranges["VegetationType"],
		other.LandSize / ranges["LandSize"],
	}

	//After normalizing, calculate the similarity using the Cosine function
    similarity, _ := Cosine(idealVector, otherVector)
    return similarity
}

## Recommend Locations

Lastly, create a function that calculates a recommendation rating for all locations, based on one given location name. <br>
This function returns a sorted array of structs that contain the Location struct and similarity float64.

In [ ]:
// Function to recommend locations based on a given location
func recommendBasedOnLocation(locations []Location, givenLocation string) []struct {
	Location   Location
	Similarity float64
} {
	// Find the given location
	var ideal Location
	for _, location := range locations {
		if location.Name == givenLocation {
			ideal = location
			break
		}
	}
	if ideal.Name == "" {
		fmt.Printf("Location '%s' not found.\n", givenLocation)
		return nil
	}

	recommendations := []struct {
		Location   Location
		Similarity float64
	}{}

	for _, location := range locations {
		if location.Name != ideal.Name {
			similarity := calculateSimilarity(ideal, location)
			recommendations = append(recommendations, struct {
				Location   Location
				Similarity float64
			}{location, similarity})
		}
	}

	// Sort recommendations by similarity
	sort.Slice(recommendations, func(i, j int) bool {
		return recommendations[i].Similarity > recommendations[j].Similarity
	})

	return recommendations
}